![image](https://i.ytimg.com/vi/31sFY83Ho9Q/maxresdefault.jpg)
# Using SQL with Jupyter Notebooks: Analyzing CIA Factbook World Population Data.


## Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
- population — the global population.
- population_growth — the annual population growth rate, as a percentage.
- area — the total land and water area.

In this project, we'll use SQL in Jupyter Notebook to analyze data from this database. You can download the `SQLite factbook.db` database by clicking [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db). You should save the database file in the same directory you have your notebook.

### Installing `ipython-sql`
To run SQL code in Jupyter, we need first to install `ipython-sql` running the code below in a code cell in Jupyter. This needs to be done only once in your system:

`!conda install -yc conda-forge ipython-sql`

### Connecting your Jypyter Notebook to the Database File
We'll run the following code to connect our Jupyter Notebook to the database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

To run SQL queries, we add `%%sql` on its own line to the start of our query.

## Information on the Tables in the Database
The code below shows the Tables included in the database.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Above, we see that there is a table in the database called `facts`. 

## Exploring the Table
Print the first 5 rows.

In [9]:
%%sql
SELECT *
  FROM facts
 LIMIT 5; 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Description of the table main columns:
- name — the name of the country.
- area— the country's total area (both land and water).
- area_land — the country's land area in square kilometers.
- area_water — the country's waterarea in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people.

## Calculating Summary Statistics
Next, we calculate the minimum and maximum population and population growth rates.

In [10]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, 
MIN(population_growth) AS min_pop_gr, MAX(population_growth) AS max_pop_gr
  FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_gr,max_pop_gr
0,7256490011,0.0,4.02


It seems odd that there is a country with population 0 and another one with more than 7 billion people. Let's further explore these countries.

## Checking the Countries with Smallest and Largest Populations

In [22]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [23]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


As we see above, the population 0 refers to Antarctica and population of 7 billion from a row related to the whole world. Next, we will recalculate the summary statistics excluding the row for the whole world.

## Recalculating Summary Statistics Removing the Total World Row

In [30]:
%%sql
SELECT MIN(population) AS min_pop, MAX(population) AS max_pop, 
MIN(population_growth) AS min_pop_gr, MAX(population_growth) as max_pop_gr
  FROM facts
 WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_gr,max_pop_gr
0,1367485388,0.0,4.02


In [27]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_pop, ROUND(AVG(area), 2) AS avg_area
  FROM facts
 WHERE name != 'World'

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.57,555093.55


Now, the country with the largest population has 1 billion people, what seems to make sense.

## Find Countries with Population Above Average and Area Below Average

In [34]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World') 
   AND area < (SELECT AVG(area) FROM facts WHERE name != 'World')
 ORDER BY population DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Countries with the Largest Population and the ones with the Highest Growth Rates

In [37]:
%%sql
SELECT name, population
  FROM facts
  WHERE name <> 'World'
  ORDER BY population DESC
  LIMIT 3;

 * sqlite:///factbook.db
Done.


name,population
China,1367485388
India,1251695584
European Union,513949445


In [38]:
%%sql
SELECT name, population_growth
  FROM facts
  WHERE name <> 'World'
  ORDER BY population_growth DESC
  LIMIT 3;

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28


## Countries with the Highest Ratio Between Water and Land

In [41]:
%%sql
SELECT *
  FROM facts
 WHERE name <> 'World' AND area_water / area_land = (SELECT MAX(area_water / area_land) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None


In [44]:
%%sql
SELECT name, area_water, area_land
  FROM facts
 WHERE area_water > area_land

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Countries with the Highest Absolute Population Growth Expected for Next Year

In [60]:
%%sql
SELECT name, ROUND(population * population_growth / 100, 2) AS abs_growth_nextyear
  FROM facts
 WHERE name <> 'World'
 ORDER BY abs_growth_nextyear DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,abs_growth_nextyear
India,15270686.12
China,6153684.25
Nigeria,4448270.37
Pakistan,2906653.37
Ethiopia,2874562.17


## Countries whose Death Rate is Higher than Birth Rate

In [58]:
%%sql
SELECT name, ROUND(death_rate - birth_rate, 2) AS net_death_birth
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY net_death_birth DESC;

 * sqlite:///factbook.db
Done.


name,net_death_birth
Bulgaria,5.52
Serbia,4.58
Latvia,4.31
Lithuania,4.17
Ukraine,3.74
Hungary,3.57
Germany,2.95
Slovenia,2.95
Romania,2.76
Croatia,2.73


In the table above, we see that many countries in Eastern Europe have death rates that are higher than birth rates.

## Population per Area Ratio

In [59]:
%%sql
SELECT name, population, area, population/area 
 FROM facts
ORDER BY population/area DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,area,population/area
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191


Small countries as Macao and Monaco are the ones with the highest population/area ratio.